In [12]:
# Necessary imports
import pandas as pd
import numpy as np
from PIL import Image
import glob
import subprocess
import os
import fileinput
import math

In [2]:
# This function inserts a Header line in the CSV file, so the columns can be identified easier
def insert_header(file):
    for linenum,line in enumerate( fileinput.FileInput(file,inplace=1) ):
        if linenum==0 :
            #This line below will be the CSV Header line
            print ("Track,Time,Event,Channel,Note,Vel,Num1,Num2")
            print (line.rstrip())
        else:
            print (line.rstrip())

In [3]:
# This function searches and returns notes which are played in the same time (only those notes which appear after index "begin" in the CSV file sorted by Time attribute)
def findNotes(begin = 0):
    # list of indexes of the position of notes in the CSV file
    indexes = []
    time = -1
    n_tempo = tempo
    # Firstly we find the time value when the notes are played
    for i in range(begin, s_data.shape[0]):
        # if in the mean time we found a Tempo event then let's return the new music tempo also
        if(s_data["Event"][i] == "Tempo"):
            n_tempo = int(s_data["Channel"][i])
        # the Note_on_c event indicates the starting of a note in the music (only if it's velocity is not null)
        if(s_data["Event"][i] == "Note_on_c" and int(s_data["Vel"][i]) != 0):
            indexes.append(i)
            time = s_data["Time"][i]
            break
    # if we haven't found any notes then we return the empty list
    if not indexes:
        return indexes, n_tempo
    # Find all the other notes which are played in the same time as the first one we found in this iteration (time value saved in variable "time" in the for loop above)
    for i in range(indexes[0] + 1, s_data.shape[0]):
         # if in the mean time we found a Tempo event then let's return the new music tempo also
        if(s_data["Event"][i] == "Tempo"):
            n_tempo = int(s_data["Channel"][i])
        # the Note_on_c event indicates the starting of a note in the music (only if it's velocity is not null)
        if(s_data["Event"][i] == "Note_on_c" and int(s_data["Vel"][i]) != 0 and s_data["Time"][i] == time):
            indexes.append(i)
        # if the current time is higher than the value stored in variable "time" then it is sure 
        # that we won't find any more notes played in the same time as the notes found in this iteration before
        if(s_data["Event"][i] == "Note_on_c" and int(s_data["Vel"][i]) != 0 and s_data["Time"][i] > time):
            break
    return indexes, n_tempo

# This function searches for the end of a currently played note
def findEndNote(note, begin = 0):
    n_tempo = tempo
    for i in range(begin, s_data.shape[0]):
        if(s_data["Event"][i] == "Tempo"):
            n_tempo = int(s_data["Channel"][i])
        # The Note_off_c or Note_on_c with 0 Velocity indicates the end of a note
        if((s_data["Event"][i] == "Note_on_c" or s_data["Event"][i] == "Note_off_c") and int(s_data["Note"][i]) == int(note) and int(s_data["Vel"][i]) == 0):
            return i, n_tempo
    return -1, n_tempo

In [10]:
# It is possible to add a Gaussian noise to the notes ( 0 expected value, 1 variation)
def add_noise(note):
    noise = int(np.random.normal(0,1,1))
    return note + noise

# This function saves an image generated from the last batch (10 seconds) of .midi file
def save_img(a_img, idx, fname):
    a_img = a_img.astype(np.uint8)
    image = Image.fromarray(a_img, mode = 'RGBA')
    image.save('C:/NHF/images/{0}_{1}.png'.format(fname, idx))
    #image.show()

In [11]:
path = 'C:\\NHF\data'
i = 0
# Find all .midi files in the given directory and convert them first into CSV files, after that the CSVs into .png files
for filename in glob.glob(path + '/*.midi'):
    print(filename)
    midi_name = filename.split("\\")[3]
    name = midi_name.split(".")[0]
    # This command launches a linux subsystem and runs a midicsv command which converts a .midi file into .csv file 
    cmd = 'bash -c "midicsv ./data/{0} actual.csv"'.format(midi_name)
    sp = subprocess.Popen(cmd)
    sp.communicate()
    # Inserting the appropiate Header into the first line of .csv
    insert_header("actual.csv")
    data = pd.read_csv("actual.csv", engine = 'python')
    
    # This value stores the number of MIDI clocks pulses per quarter note
    division = int(data["Vel"][0])
    
    # Trimming the spaces of the string values
    data["Event"] = data["Event"].str.strip()
    #Sorting the .csv file by the Time parameter of each event
    s_data = data.sort_values(by = "Time")
    if(isinstance(s_data["Vel"][0], str)):
        s_data["Vel"] = s_data["Vel"].str.strip()
    if(isinstance(s_data["Note"][0], str)):
        s_data["Note"] = s_data["Note"].str.strip()
    if(isinstance(s_data["Channel"][0], str)):
        s_data["Channel"] = s_data["Channel"].str.strip()
    s_data.to_csv("test.csv", index = False)
    s_data = pd.read_csv("test.csv", engine = 'python')
    tempo = 0
    
    # Finding the first Tempo event to store the value of the initial tempo
    for i in range(s_data.shape[0]):
         if(s_data["Event"][i] == "Tempo"):
            tempo = int(s_data["Channel"][i])
            break
            
    # The value of the possibly lowest note (value of notes on an 88 key piano are between 21 - 109)
    min_note = 21
    # One pixel on the image means 0.05 time slice in the music
    pixel_time = 0.05
    tot_time = 0
    start_pxl = 0
    # Finding the first couple of notes which are played in the same time
    index, new_tempo = findNotes()
    # In img array we will store the values of pixels of the generated image
    img = np.zeros(shape = (88,200, 4))
    # The opacity is 255 for all pixels in the beginning
    img[:,:,3] = np.ones(shape = (88,200))*255
    img_idx = 0
    # While there are notes to process in csv file do the following instructions:
    while(index):
        # Counting the time difference between the current played notes and those which were began to be played in the past iteration
        time_diff =  int(s_data["Time"][index[0]]) - tot_time
        # According to Official MIDI File Format Documentation Beats per minute is defined as:
        BPM = 60000000 / tempo
        # clock_time stores how many seconds does last one MIDI clock pulse
        clock_time = 60.0/(BPM*division)
        # start_pxl stores on which pixel should appear the currently played note (taking into account also the tempo of the music)
        start_pxl += int(np.round(clock_time*time_diff/0.05))
        # Refreshing the tempo if it has been changed while searching for currently played notes
        tempo = new_tempo
        # Updating the last ote batch time value
        tot_time = int(s_data["Time"][index[0]])
        # After 10 seconds (10/0.05s = 200) we save the actual img array and begin generating another one from an empty img array
        if(start_pxl >= 200):
            start_pxl = 0
            save_img(img, img_idx, name)
            img = np.zeros(shape = (88,200, 4))
            img[:,:,3] = np.ones(shape = (88,200))*255
            img_idx += 1
        # Processing the notes of the last batch
        for i in range(len(index)):
            n = s_data["Note"][index[i]]
            # end stores the time value (in MIDI clock pulses) when the note has stopped being played
            end, new_tempo = findEndNote(note = n, begin = index[i] + 1)
            # Update Tempo if we found a Tempo event in the meantime
            tempo = new_tempo
            
            # The length of a note will be a real number between 0 and 1 as it is defined in music theory
            length = ((s_data["Time"][end] - s_data["Time"][index[i]])/(division*4))
        
            BPM = 60000000 / tempo
            
            # nr_pxl_q indicates how many pixel long is one quarter note on the generated image
            nr_pxl_q = 60/(pixel_time*BPM)
            # From the value of nr_pxl_q it is easy to determine how many pixel long is the currently played note
            nr_pxl = np.round(nr_pxl_q*length/0.25)
            
            # We used tangent-hyprbolic to rescale the length of a note, because on a linear scale it is harder to notice the difference between very short notes
            # and we also want the length of the notes to be easily recognizable on the generated image
            scaled_length = np.tanh(3*length)
            # the maximum value of tempo is 16.777.215 so we rescale the value of tempo in the following way:
            scaled_tempo = tempo/16777215
            
            # We use all the four channel of one pixel in the following way:
            
            # The first (R) channel stores information about the length of a note (we used the scaled_value here)
            img[87 - (int(n) - min_note),start_pxl,0] = np.round(255 - scaled_length*250) 
            # The second (G) channel stores by which instrument is played the current note
            img[87 - (int(n) - min_note),start_pxl,1] = 0
            # The third (B) channel stores the Velocity of a note (how loud the note is)
            img[87 - (int(n) - min_note),start_pxl,2] = 100 
            # The last channel (A) stores the scaled actual tempo value
            img[87 - (int(n) - min_note),start_pxl,3] = np.round(255 - scaled_tempo*255)   #tempo
        index, new_tempo = findNotes(begin = index[-1] + 1)
    # In the end of the whole process we save also the last batch (even if it is not exactly 10 seconds)
    save_img(img, img_idx, name)
    # After every midi-csv conversion we delete the generated .csv file for saving memory.
    os.remove("actual.csv")
    os.remove("test.csv")

C:\NHF\data\MIDI-Unprocessed_Chamber1_MID--AUDIO_07_R3_2018_wav--2.midi
C:\NHF\data\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1.midi
C:\NHF\data\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--3.midi
C:\NHF\data\MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi
C:\NHF\data\MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--2.midi
C:\NHF\data\MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--3.midi
C:\NHF\data\MIDI-Unprocessed_Chamber4_MID--AUDIO_11_R3_2018_wav--1.midi
C:\NHF\data\MIDI-Unprocessed_Chamber4_MID--AUDIO_11_R3_2018_wav--3.midi
C:\NHF\data\MIDI-Unprocessed_Chamber5_MID--AUDIO_18_R3_2018_wav--1.midi
C:\NHF\data\MIDI-Unprocessed_Chamber5_MID--AUDIO_18_R3_2018_wav--2.midi
C:\NHF\data\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1.midi
C:\NHF\data\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--2.midi
C:\NHF\data\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--3.midi
C:\NHF\data\MIDI-Unprocessed_Recital1-3_MID--AUDIO_01_R1_2018_wa